In [1]:
import networkx as nx
import osmnx as ox
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
#Get the edge list
edgelist_fn = '../data/processed/BaseEdgeList.csv'
df_edgeList = pd.read_csv(edgelist_fn,
                          header = 1,
                          names = ['From_node','To_node','Site_yield_scf_h'],
                          dtype={'From_node':str,'To_node':str,'Site_yield_scf_h':np.float64})
df_edgeList.sample(5)

,From_node,To_node,Site_yield_scf_h
273,274,274_T,NaN
319,320,321,4.710241e+07
1638,1639,1640,1.116783e+07
1382,1383,1383_T,1.468685e+07
2300,2301,1985,4.069850e+06


In [3]:
#Create the graph
G = nx.MultiDiGraph()

In [4]:
#Add the edges
for i,r in df_edgeList.iterrows():
    G.add_edge(r.From_node,r.To_node,cost=r.Site_yield_scf_h)

#### Compute total upstream biogas potential 
Iterate through each "from" node and find all its upstream nodes and sum their collective biogas potential values

In [5]:
#Iterate through each to_node
for from_node in df_edgeList.From_node.unique():
    #Reset the cost variable
    total_yield = 0
    #Get the upstream nodes for the current node
    up_nodes = nx.ancestors(G,from_node)
    #Append the node itself
    up_nodes.add(from_node)
    #Iterate through the to_node's ancestors
    for up_node in up_nodes:
        #Accumulate the total yield
        total_yield += df_edgeList.query("From_node == '{}'".format(up_node))['Site_yield_scf_h'].sum()
    df_edgeList.loc[df_edgeList.From_node == from_node,'SumYield'] = total_yield

#Compute pipe diameter from throughput
df_edgeList['PipeSize_in'] = df_edgeList['SumYield']**0.394*0.051
#Compute pipe cost from throughput 
df_edgeList['PipeCost'] = df_edgeList['PipeSize_in']**0.485*10829 
#Compute return
df_edgeList['Return_15yr'] = df_edgeList['PipeCost'] / (df_edgeList['SumYield'] * 15)

In [6]:
#Save edge list to a filename
edgeCSV_filename =  '../scratch/Biogas_12May2020.csv'
df_edgeList[['From_node','Site_yield_scf_h', 'SumYield', 'PipeSize_in',
            'PipeCost', 'Return_15yr']].to_csv(edgeCSV_filename,index=False)

KeyError: "['Return_15yr', 'PipeSize_in', 'PipeCost'] not in index"

In [7]:
df_edgeList['SumYield'] = df_edgeList.SumYield.astype(int)
df_edgeList.head()

,From_node,To_node,Site_yield_scf_h,SumYield
0,1,2,NaN,195479397
1,2,3,NaN,213198120
2,3,4,NaN,320304814
3,4,5,NaN,324405971
4,5,6,NaN,376836073


In [8]:
#Read in segments and join cumulative data
gdf_Segments = gpd.read_file('../data/processed/BasePipeline.shp')
gdf_Segments = gdf_Segments.merge(df_edgeList,left_on='from_id',right_on='From_node',how='left')
gdf_Segments.to_file('../scratch/segments.shp')